In [1]:
import os
from sqlalchemy import create_engine
import pandas as pd

project_dir = os.path.dirname(os.getcwd())
database_file = f"sqlite:///{os.path.join(os.getcwd(), 'weather.db')}"

engine = create_engine(database_file)
raw_df = pd.read_sql_table('forecast', con=engine)

In [2]:
def select_columns(df):
    remove = [ # check for these columns to remove
        'id',
        'cod',
        'base',
        'weather_icon',
        'sys_id',
        'sys_message',
        'sys_type',
        'dt',
        'timezone',
        'weather_id',
        'weather_main',
        'weather_description',
        'main_sea_level',
        'main_grnd_level',
        'sys_country',
        'name']

    try:
        keep_col = [i for i, col in enumerate(df.columns) if col not in remove]
    except:
        print('Data does not match what is expected.')

    return df.iloc[:, [*keep_col]]

def clean_data(df):
    select_columns(df)
    df['positive'] = df['positive'].astype('int')
    df.fillna(0, inplace=True)
    

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer

#df = clean_data(raw_df)
df = pd.concat([raw_df, raw_df, raw_df, raw_df, raw_df, raw_df], axis=0)

transformer = FunctionTransformer(clean_data)
logreg = LogisticRegression(solver="lbfgs")

pipe = make_pipeline(transformer, logreg)

X = df.drop('positive', axis=1)
y = df.positive


In [23]:

cross_val_score(pipe, X, y, cv=5, scoring="accuracy").mean()

C:\Users\njrosati\miniconda3\lib\site-packages\sklearn\preprocessing\_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
C:\Users\njrosati\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: could not convert string to float: 'Clear'

In [0]:
logreg.fit(X, y)

In [0]:
logreg.predict(X[1:2])